# Chapter 3: Coding attention mechanisms

## 3.3 Attending to different parts of the input with self-attention

### 3.3.1 A simple self-attention mechanism without trainable weights

In [ ]:
import torch

In [ ]:
# embedding vectors

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)

In [ ]:
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

attn_scores_2

In [ ]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
attn_weights_2_tmp, attn_weights_2_tmp.sum()

In [ ]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

In [ ]:
attn_weights_2_naive = softmax_naive(attn_scores_2)
attn_weights_2_naive, attn_weights_2_naive.sum()

In [ ]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
attn_weights_2, attn_weights_2.sum()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(attn_weights_2_tmp.shape[0]), attn_weights_2_tmp,label='sum')
plt.plot(range(attn_weights_2.shape[0]), attn_weights_2,label='softmax')
plt.legend()
plt.show()

In [ ]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)

for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

context_vec_2

### 3.3.2 Computing attention weights for all input tokens

In [ ]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

attn_scores

In [ ]:
attn_scores = inputs @ inputs.T
attn_scores

In [ ]:
attn_weights = torch.softmax(attn_scores, dim=-1)
attn_weights

In [ ]:
all_context_vecs = attn_weights @ inputs
all_context_vecs

## 3.4 Implementing self-attention with trainable weights

### 3.4.1 Computing the attention weights step by step

In [ ]:
x_2 = inputs[1] # single element from input embedding sequence
d_in = inputs.shape[1] # input embedding size
d_out = 2 # output embedding size

In [ ]:
torch.manual_seed(123)

In [ ]:
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [ ]:
x_2.shape, W_query.shape

`x_2 @ W_query` is the same as $x_2^{T}W_{query}$

In [ ]:
# PyTorch treats a 1D vector of shape (3,) as a row vector when used on the left of @.
query_2 = x_2 @ W_query # (1x3) x (3x2) => 1x2 => torch.Size([2])
query_2.shape

In [ ]:
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

In [ ]:
keys = inputs @ W_key
values = inputs @ W_value

In [ ]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
attn_score_22

In [ ]:
attn_scores_2 = query_2 @ keys.T
attn_scores_2

In [ ]:
d_k = keys.shape[-1]
d_k

In [ ]:
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
attn_weights_2

In [ ]:
context_vec_2 = attn_weights_2 @ values
context_vec_2

### 3.4.2 Implementing a compact self-attention Python class

In [ ]:
import torch.nn as nn

In [ ]:
# NOTE: this is of course scaled dot product self-attention

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
    
    def forward(self, x):
        queries = x @ self.W_query
        keys = x @ self.W_key
        values = x @ self.W_value

        attn_scores = queries @ keys.T # omega

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values

        return context_vec

In [ ]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
sa_v1(inputs)

In [ ]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T # omega

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values

        return context_vec

In [ ]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
sa_v2(inputs)

**Exercise 3.1 Comparing SelfAttention_v1 and SelfAttention_v2**

Note that `nn.Linear` in `SelfAttention_v2` uses a different weight initialization scheme as `nn.Parameter(torch.rand(d_in, d_out))` used in `SelfAttention_v1`, which causes both mechanisms to produce different results. To check that both implementations, `SelfAttention_v1` and `SelfAttention_v2`, are otherwise similar, we can transfer the weight matrices from a `SelfAttention_v2` object to a `SelfAttention_v1`, such that both objects then produce the same results.

Your task is to correctly assign the weights from an instance of `SelfAttention_v2` to an instance of `SelfAttention_v1`. To do this, you need to understand the relationship between the weights in both versions. (Hint: nn.Linear stores the weight matrix in a transposed form.) After the assignment, you should observe that both instances produce the same outputs.

In [ ]:
# TODO: Somehow use torch load, save, and transpose...